In [1]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms

In [2]:
# basic自动求导例子1
# 创建张量
x = torch.tensor(1.,requires_grad=True)
w = torch.tensor(2.,requires_grad=True)
b = torch.tensor(3.,requires_grad=True)

# 计算图
y = w * x + b    # y = 2 * x + 3

# 计算梯度
y.backward()

In [3]:
print(f"x梯度：{x.grad},w梯度：{w.grad},b梯度：{b.grad}。")

x梯度：2.0,w梯度：1.0,b梯度：1.0。


In [4]:
# basic自动求导例子2
# 创建张量
x = torch.randn(10,3)
y = torch.randn(10,2)

x,y


(tensor([[ 0.0161, -1.2447,  0.4456],
         [ 0.6961,  0.0405, -1.0334],
         [ 0.8368,  0.0649,  1.0216],
         [ 0.3938,  0.2129,  0.4330],
         [ 2.4811,  0.1600, -0.5196],
         [ 0.7483, -1.4351,  0.4341],
         [ 1.1426,  0.0049, -1.3054],
         [-0.7675, -0.0376, -0.2953],
         [-0.1043,  0.5747,  0.2419],
         [ 0.9974, -1.1863,  1.2229]]),
 tensor([[-0.7404, -0.4340],
         [-0.3005, -0.5322],
         [ 0.4864,  0.7186],
         [ 1.8049,  0.2626],
         [ 1.0370,  0.1492],
         [ 0.2374, -0.0412],
         [ 0.9889,  0.6082],
         [ 0.3513,  0.2129],
         [-0.5119,  1.7907],
         [ 0.2324, -0.3144]]))

In [5]:
# 定义全连接层
linear = nn.Linear(3,2)

In [6]:
linear.weight

Parameter containing:
tensor([[ 0.1015, -0.4284, -0.2599],
        [ 0.5061, -0.3982, -0.5315]], requires_grad=True)

In [7]:
linear.bias

Parameter containing:
tensor([ 0.3696, -0.1182], requires_grad=True)

In [8]:
linear

Linear(in_features=3, out_features=2, bias=True)

In [9]:
# 定义损失和优化
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(),lr=0.01)

In [10]:
# 前向传播
output = linear(x)

In [11]:
# 计算损失
loss = criterion(output,y)

In [12]:
loss.item()

0.9240363836288452

In [13]:
# 反向传播
loss.backward()

In [14]:
linear.weight.grad

tensor([[-0.0368, -0.3487, -0.0012],
        [ 0.5162, -0.3502, -0.3116]])

In [15]:
linear.bias.grad

tensor([0.1815, 0.0447])

In [16]:
optimizer.step()

In [17]:
linear.weight

Parameter containing:
tensor([[ 0.1018, -0.4249, -0.2599],
        [ 0.5010, -0.3947, -0.5284]], requires_grad=True)

In [18]:
linear.bias

Parameter containing:
tensor([ 0.3677, -0.1187], requires_grad=True)

In [19]:
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.item())

loss after 1 step optimization:  0.9176262617111206


In [20]:
# Create a numpy array.
x = np.array([[1, 2], [3, 4]])

# Convert the numpy array to a torch tensor.
y = torch.from_numpy(x)

# Convert the torch tensor to a numpy array.
z = y.numpy()

In [21]:
x,y,z

(array([[1, 2],
        [3, 4]]),
 tensor([[1, 2],
         [3, 4]]),
 array([[1, 2],
        [3, 4]]))

In [22]:
# 导入数据
train_dataset = torchvision.datasets.CIFAR10(root='../../../../data/',
                                            train=True,
                                            transform=transforms.ToTensor(),
                                            download=True)

image,label = train_dataset[0]

In [23]:
image.size,label

(<function Tensor.size>, 6)

In [24]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64, 
                                           shuffle=True)

# When iteration starts, queue and thread start to load data from files.
data_iter = iter(train_loader)

# Mini-batch images and labels.
images, labels = data_iter.next()

# Actual usage of the data loader is as below.
for images, labels in train_loader:
    # Training code should be written here.
    pass

AttributeError: '_SingleProcessDataLoaderIter' object has no attribute 'next'

In [ ]:
# You should build your custom dataset as below.
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self):
        # TODO
        # 1. Initialize file paths or a list of file names. 
        pass
    def __getitem__(self, index):
        # TODO
        # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open).
        # 2. Preprocess the data (e.g. torchvision.Transform).
        # 3. Return a data pair (e.g. image and label).
        pass
    def __len__(self):
        # You should change 0 to the total size of your dataset.
        return 0 

# You can then use the prebuilt data loader. 
custom_dataset = CustomDataset()
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                           batch_size=64, 
                                           shuffle=True)


# ================================================================== #
#                        6. Pretrained model                         #
# ================================================================== #

# Download and load the pretrained ResNet-18.
resnet = torchvision.models.resnet18(pretrained=True)

# If you want to finetune only the top layer of the model, set as below.
for param in resnet.parameters():
    param.requires_grad = False

# Replace the top layer for finetuning.
resnet.fc = nn.Linear(resnet.fc.in_features, 100)  # 100 is an example.

# Forward pass.
images = torch.randn(64, 3, 224, 224)
outputs = resnet(images)
print (outputs.size())     # (64, 100)


# ================================================================== #
#                      7. Save and load the model                    #
# ================================================================== #

# Save and load the entire model.
torch.save(resnet, 'model.ckpt')
model = torch.load('model.ckpt')

# Save and load only the model parameters (recommended).
torch.save(resnet.state_dict(), 'params.ckpt')
resnet.load_state_dict(torch.load('params.ckpt'))
